In [52]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
import os
from openai import AzureOpenAI
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://<your-endpoint>.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

In [62]:
# # gets the API Key from environment variable AZURE_OPENAI_API_KEY
# client = AzureOpenAI(
#     api_version=os.environ.get("AZURE_OPENAI_VERSION"),
#     api_key=os.getenv("AZURE_OPENAI_KEY"),
#     # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
#     azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
# )

# completion = client.chat.completions.create(
#     model=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),  # e.g. gpt-35-instant
#     messages=[
#         {
#             "role": "user",
#             "content": "How do I output all files in a directory using Python?",
#         },
#     ],
# )
# print(completion.choices[0].message.content)

In [42]:
openai_client_embeddings = AzureOpenAI(
                api_key=os.getenv("AZURE_OPENAI_KEY"),
                azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_embeddings", "embed"),
                api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
                azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT","https://<your-endpoint>.openai.azure.com/")
)
# use this for embeddings

In [43]:
create_args = {
            "model": "text-embedding-ada-002",
             "input": "hey there",
                    }
res2 = openai_client_embeddings.embeddings.create(**create_args)

In [45]:
ex = res2.data[0].embedding

In [46]:
print(ex)

[-0.037249911576509476, 0.005593954119831324, -0.016141628846526146, -0.03215391933917999, -0.025260096415877342, 0.01757730171084404, -0.011744069866836071, -0.016115760430693626, -0.004313488025218248, -0.006521968171000481, 0.027368338778614998, 0.0014785175444558263, -0.018870702013373375, -0.009040864184498787, -0.0002540317946113646, -0.014925832860171795, 0.026799242943525314, -0.004536599852144718, 0.025867994874715805, -0.010592943988740444, 0.0004939170321449637, 0.000510488694999367, -0.006269755307585001, -0.02176791802048683, -0.0202805083245039, -0.011220242828130722, 0.02352694235742092, -0.027627019211649895, 0.016232166439294815, -0.02364334836602211, -0.00025544644449837506, 0.006680409889668226, -0.013503093272447586, -0.008782184682786465, -0.012526576407253742, -0.03070531040430069, -0.004817914217710495, -0.0048308479599654675, 0.018094662576913834, -0.006473465822637081, 0.032050445675849915, -0.011634130962193012, -0.005952872335910797, -0.018637889996170998, -0

In [9]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [11]:
len(all_splits)

66

In [47]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=openai_client_embeddings)

AttributeError: 'AzureOpenAI' object has no attribute 'embed_documents'

In [78]:
type(all_splits[0])

langchain_core.documents.base.Document

In [77]:
all_splits[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 8}

In [99]:
metadata = [split.metadata for split in all_splits]

In [84]:
response = openai_client_embeddings.embeddings.create(
    input=[split.page_content for split in all_splits],
    model="text-embedding-ada-002",
)

In [27]:
input_filex = [split.page_content for split in all_splits]
create_args = {
            "model": "text-embedding-ada-002",
             "input": input_filex
                    }
res = openai_client_embeddings.embeddings.embed_documents(**create_args)

AttributeError: 'Embeddings' object has no attribute 'embed_documents'

In [95]:
type(res)

openai.types.create_embedding_response.CreateEmbeddingResponse

In [117]:
len(input_filex)

66

TypeError: FAISS.__init__() missing 1 required positional argument: 'embedding_function'

In [ ]:
# vectorstore1.add_embeddings(res.data[0], metadata)

In [141]:
# import chromadb
# CHROMA_DATA_PATH = "chroma_data/"
# EMBED_MODEL = "text-embedding-ada-002"
# COLLECTION_NAME = "demo_docs"

In [147]:
# embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
# ...     model_name=EMBED_MODEL
# ... )

# >>> collection = client.create_collection(
# ...     name=COLLECTION_NAME,
# ...     embedding_function=embedding_func,
# ...     metadata={"hnsw:space": "cosine"},
# ... )


# collection.add(
# ...     documents=documents,
# ...     ids=[f"id{i}" for i in range(len(documents))],
# ...     metadatas=[{"genre": g} for g in genres]
# ... )

Note: you may need to restart the kernel to use updated packages.


In [154]:
def azure_openai_embed_function(text):
    # Call the Azure OpenAI service to create embeddings
    embedding_response = openai_client_embeddings.embeddings.create(
        model="text-embedding-ada-002",
        input=[text]
    )
    # Extract and return the embeddings from the response
    embeddings = embedding_response.data[0]
    return embeddings

In [54]:
hf_token = os.getenv("HF_TOKEN")
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
        api_key=hf_token, model_name="BAAI/bge-base-en-v1.5"
    )

In [55]:
vectorstore1 = FAISS.from_documents(documents=all_splits, embedding=embeddings)

In [74]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [75]:
question = "What are the approaches to Task Decomposition?"

In [76]:
retrieved_docs = retriever.invoke(question)

In [77]:
print(retrieved_docs)

[Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2192}), Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decompositio

In [78]:
prompt1 = """You are a helpful assisntant and you get question and also retrieved answer from the document.
please provide the answer to the question. In a few sentences. Do not not write something that is not in the document."""
prompt = question + "\n\n" + retrieved_docs[0].page_content

In [79]:
print(prompt1+prompt)

You are a helpful assisntant and you get question and also retrieved answer from the document.
please provide the answer to the question. In a few sentences. Do not not write something that is not in the document.What are the approaches to Task Decomposition?

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


In [80]:
default_llm.invoke(prompt)